# users table에 패스워드 넣기

In [39]:
import json
import pymysql

In [40]:
with open('mysql.json','r') as file:
    config_str = file.read()
config_str

'{\n    "host": "localhost",\n    "user": "hkuser",\n    "password": "hkpass",\n    "database": "world",\n    "port": 3306\n}'

In [41]:
config = json.loads(config_str)
config

{'host': 'localhost',
 'user': 'hkuser',
 'password': 'hkpass',
 'database': 'world',
 'port': 3306}

In [42]:
conn = pymysql.connect(            
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database =  config['database'],
    port =  config['port']
)

- 패스워드 암호화

In [43]:
# 파이썬에서 기본으로 제고하는 모듈 활용
import hashlib
import base64

- Secure Hash Algorithm(SHA-256) : 단방향 알고리즘

In [44]:
pwd = '1234'
pwd_sha256 = hashlib.sha256(pwd.encode())
pwd_sha256.digest()

b'\x03\xacgB\x16\xf3\xe1\\v\x1e\xe1\xa5\xe2U\xf0g\x956#\xc8\xb3\x88\xb4E\x9e\x13\xf9x\xd7\xc8F\xf4'

- Base64로 인코딩

In [45]:
base64.b64encode(pwd_sha256.digest())

b'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [46]:
hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
hashed_pwd

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

In [47]:
len(hashed_pwd)

44

In [48]:
# 패스워드를 생성하는 함수만들기
def gen_pwd(pwd):
    pwd_sha256 = hashlib.sha256(pwd.encode())
    hashed_pwd = base64.b64encode(pwd_sha256.digest()).decode('utf-8')
    return hashed_pwd

### users table을 읽어서 pwd 열에 패스워드를 채우기

In [49]:
sql = 'SELECT uid FROM users;'
cur = conn.cursor()
cur.execute(sql)
results = cur.fetchall()
results

(('admin',),
 ('djy',),
 ('eskim',),
 ('gdhong',),
 ('jbpark',),
 ('vincenzo',),
 ('wjlee',))

In [50]:
# uid+'12' 한 password 생성
sql = 'UPDATE users SET pwd=%s where uid=%s;'
cur = conn.cursor()
for row in results:
    uid = row[0]
    pwd = gen_pwd(uid+'12')             # 비밀번호열에 12추가  
    cur.execute(sql, (pwd, uid))        # 파라미터에 pwd, uid를 준다
conn.commit()

### 로그인

- 1) uid OK, pwd OK

In [51]:
uid, pwd = 'admin', 'admin12'
sql = "SELECT uid, pwd FROM users WHERE uid=%s;"
cur = conn.cursor()
cur.execute(sql, (uid,))
result = cur.fetchone()     # 하나만 가져오기, 첫번쨰데이터는 uid값
db_pwd = result[1]          # 두번째 element 뽑음
db_pwd

'EUZjqxlO3LP2HUCYg85K5sPC+YVBlAlaU4UBHRW+y+8='

In [52]:
# hashed password 구하기
# 위와 아래 결과가 같은경우
hashed_pwd = gen_pwd(pwd)
hashed_pwd

'EUZjqxlO3LP2HUCYg85K5sPC+YVBlAlaU4UBHRW+y+8='

- 2) uid OK, pwd BAD

In [53]:
pwd = '1234'
hashed_pwd = gen_pwd(pwd)
hashed_pwd
# pwd 가 다르다

'A6xnQhbz4Vx2HuGl4lXwZ5U2I8iziLRFnhP5eNfIRvQ='

- 3) uid BAD

In [54]:
uid = 'fool'

In [55]:
sql = "SELECT uid, pwd FROM users WHERE uid=%s;"
cur = conn.cursor()
cur.execute(sql, (uid,))

0

In [56]:
result = cur.fetchone()

In [57]:
result              # 확인결과 result에 아무것도 없음

- 1),2),3)을 하나로 합치기

In [58]:
# uid랑 pwd를 다른거로 바꾸면 밑에 결과가 달라진다 uid랑 pwd를 다른거로 바꾸면 밑에 결과가 달라진다
uid, pwd = 'admin','admin12'           

In [59]:
sql = "SELECT uid, pwd FROM users WHERE uid=%s and is_deleted=0;"
cur = conn.cursor()
cur.execute(sql, (uid,))
result = cur.fetchone()
if result:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('correct')
    else:
        print('incorrect password')
else:
    print('bad uid')

correct


In [60]:
cur = conn.cursor()
cur.execute(sql, (uid,))
result = cur.fetchone()
try:
    db_pwd = result[1]
    hashed_pwd = gen_pwd(pwd)
    if db_pwd == hashed_pwd:
        print('correct')
    else:
        print('incorrect password')
except:
    print('bad uid')

correct


In [61]:
# 다끝났으면 마지막 마무리로 써줘야함
cur.close()
conn.close()